In [1]:
from snowexsql.db import get_db
from snowexsql.data import ImageData, SiteData, LayerData
from rasterio.plot import show
from sqlalchemy.sql import func
import geoalchemy2.functions as gfunc
from geoalchemy2.types import Raster
from geoalchemy2.shape import to_shape
import geopandas as gpd
from snowexsql.conversions import raster_to_rasterio
from snowexsql.conversions import points_to_geopandas, query_to_geopandas, query_to_pandas
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import pandas as pd

In [2]:
def get_bufferedPit(pit_id):
    # Distance around the pit to collect data in meters
    buffer_dist = 50

    datasets = []

    # Grab our sites details by site id
    q = session.query(LayerData).filter(LayerData.pit_id==pit_id)
    sites = q.all()

    # Grab the pit location from a single layer
    p = sites[0].geom

    # Convert the point to a pyshapely
    pit = to_shape(p)

    # Convert it to a geopandas dataframe for easy plotting
    df_pit = gpd.GeoSeries(pit)
    q = session.query(gfunc.ST_Buffer(p, buffer_dist))
    buffered_pit = q.all()[0][0]
    return buffered_pit

In [3]:
def average_amplitudes(raster):
    """
    input: 2 dimensional array of list of amplitudes over multiple rasters
    output: average amplitude over entire raster
    """
    #create a variable to average each individual raster
    single_avgs = []
    #iterate over all rasters
    for i in range(len(raster)): 
        #compute and append the mean of the raster 
        single_avgs.append(sum(dataset.read(1)[i])/len(dataset.read(1)[i]))
    
    #compute the average amplitude over all rasters
    avg_amp = sum(single_avgs)/len(single_avgs) 
    
    return avg_amp

In [4]:
# Pit Site Identifier of interest
site_name = 'Grand Mesa'

# Connect to the database we made.
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

datasets = []
amplitudes = []

# Grab our sites details by site id
#q = session.query(SiteData).filter(SiteData.site_id==site_id)
q = session.query(SiteData.site_id).filter(SiteData.site_name==site_name)
q = q.filter(SiteData.tree_canopy=="No Trees")
q = q.filter(SiteData.date>=date(2020,1,28))
q = q.filter(SiteData.date<=date(2020,3,1))
sites = q.all()
sites = [d[0] for d in sites] 
q_layer = session.query(LayerData.pit_id).filter(LayerData.site_id.in_(sites))
q_layer = q_layer.distinct()
df = query_to_pandas(q_layer, engine)
df.sort_values(by=['pit_id'],ascending=True)

# Grab the rasters, union them and convert them as tiff when done
q_raster = session.query(func.ST_AsTiff(func.ST_Union(ImageData.raster, type_=Raster)))

# Only grab rasters that are the bare earth DEM from USGS
q_raster = q_raster.filter(ImageData.type == 'insar amplitude').filter(ImageData.observers=='UAVSAR team, JPL').filter(ImageData.site_name == "Grand Mesa")

for id in df.pit_id:
    try:
        buffered_pit = get_bufferedPit(id)
        q_raster_new = q_raster.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))
        rasters = q_raster_new.all()
        dataset = raster_to_rasterio(session, rasters)[0]
        avg_amplitudes = average_amplitudes(dataset.read(1))
        amplitudes.append((id,avg_amplitudes))
    except TypeError:
        print("I failed")
        continue
print(amplitudes)

/tmp/ipykernel_3859/3189073905.py:20: SAWarning: Class ST_Buffer will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  buffered_pit = q.all()[0][0]
/tmp/ipykernel_3859/1880334317.py:34: SAWarning: Class ST_Union will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alte

[('COGM2S37_20200201', 0.0405524388824432), ('COGMSO_20200321_1006', 0.044672281846771966), ('COGM5S31_20200130', 0.10746583853418201), ('COGMCO_20200318_0825', 0.03891090907782768), ('COGM2C2_20200131', 0.019295577679073972), ('COGMSO_20200328_1630', 0.044672281846771966), ('COGMSO_20200406_0650', 0.06971825616878391), ('COGM1C7_20200131', 0.02365567432643445), ('COGM6C10_20200131', 0.02365567432643445), ('COGM1N23_20200211', 0.04076557850619622), ('COGM8N34_20200130', 0.03823637002204805), ('COGMSO_20200212_1252', 0.044672281846771966), ('COGM6S44_20200204', 0.04742229587230895), ('COGMSO_20200225_1500', 0.044672281846771966), ('COGM1N1_20200208', 0.06119450896338208), ('COGM3N22_20200128', 0.04076557850619622), ('COGMSO_20200328_0900', 0.06971825616878391), ('COGM2N12_20200131', 0.04076557850619622), ('COGM2S48_20200129', 0.031048427987713012), ('COGMSO_20200419_1600', 0.044672281846771966), ('COGM2C3_20200131', 0.019295577679073972), ('COGM5C27_20200209', 0.03164488379256625), ('CO

In [6]:
amp = pd.DataFrame(amplitudes)
amp.to_csv('amplitudes.csv')